In [1]:
import pandas as pd
import pmdarima as pmd
import xgboost as xgb

https://apisidra.ibge.gov.br/

/T/  Tabela:  	1737 - 	IPCA - Série histórica com número-índice, variação mensal e variações acumuladas em 3 meses, em 6 meses, no ano e em 12 meses (a partir de dezembro/1979)

/V/  Variável(6):

2266  IPCA - Número-índice (base: dezembro de 1993 = 100) (Número-índice) - casas decimais: padrão = 13, máximo = 13 <br/>
63  IPCA - Variação mensal (%) [janeiro 1980 a março 2024] - casas decimais: padrão = 2, máximo = 2
2263  IPCA - Variação acumulada em 3 meses (%) [março 1980 a março 2024] - casas decimais: padrão = 2, máximo = 2<br/>
2264  IPCA - Variação acumulada em 6 meses (%) [junho 1980 a março 2024] - casas decimais: padrão = 2, máximo = 2<br/>
69  IPCA - Variação acumulada no ano (%) [janeiro 1980 a março 2024] - casas decimais: padrão = 2, máximo = 2<br/>
2265  IPCA - Variação acumulada em 12 meses (%) [dezembro 1980 a março 2024] - casas decimais: padrão = 2, máximo = 2

Nível Territorial:

/N1/ Brasil(1)   Listar unidades territoriais


In [2]:
# Coleta de dados
dados_brutos = pd.read_json("https://apisidra.ibge.gov.br/values"+
                            "/t/1737/n1/all/v/2266/p/all/d/v2266%2013?formato=json")
dados_brutos.head(3)

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Brasil (Código),Brasil,Variável (Código),Variável,Mês (Código),Mês
1,1,Brasil,30,Número-índice,0.0000000076183,1,Brasil,2266,IPCA - Número-índice (base: dezembro de 1993 =...,197912,dezembro 1979
2,1,Brasil,30,Número-índice,0.0000000081223,1,Brasil,2266,IPCA - Número-índice (base: dezembro de 1993 =...,198001,janeiro 1980


In [3]:
dados = dados_brutos.rename(columns={"V": "valor", "D3C": "data"}).iloc[1:]
dados["data"] = pd.to_datetime(dados["data"], format="%Y%m")
dados = dados.set_index("data",drop=False).assign(valor=lambda x: x.valor.astype(float))
# dados = dados.query("data >= '2004-01-01'")[["valor"]].asfreq("MS")
dados = dados[dados.index >= '2004-01-01'][["data","valor"]].asfreq("MS")

dados['ipca'] = (dados.valor / dados.valor.shift(1) - 1) * 100
dados['ano']  = dados.data.apply(lambda x: x.year)
dados['mes']  = dados.data.apply(lambda x: x.month)
dados['trimestre'] = dados.data.apply(lambda x: x.quarter)
dados['ipca_lag1'] = dados.ipca.shift(1)
dados['tendencia'] = range(-1, dados.shape[0] - 1)
dados = dados.dropna()


In [4]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=dados.index,  # Usa o índice 'data' como eixo x
    y=dados['valor'],
    mode='lines',  # Define o tipo de gráfico como linha
    name='IPCA'  # Nome da série (opcional)
))

fig.update_layout(
    title='Inflação - IPCA',
    xaxis_title="data", yaxis_title="Nº índice",
    xaxis_tickformat="%Y", xaxis_dtick="M24",  # Intervalo de ticks a cada 24 meses (2 anos)
    width=600, height=300,
    annotations=[dict(
        text="Número-índice (base: dezembro de 1993 = 100) [fonte: IBGE]",
        showarrow=False,
        xref="paper", yref="paper",
        x=.75, y=1.25,  # Posição do subtítulo
        xanchor="right", yanchor="top"  # Ancoragem do subtítulo
    )]
 )

fig.show()

In [5]:
dados_teste = dados.tail(12).drop(["data", "valor"], axis = "columns")
y_teste = dados_teste.ipca
x_teste = dados_teste.drop("ipca", axis = "columns")

dados_treino = dados.query("data not in @y_teste.index").drop(
  labels = ["data", "valor"],
  axis = "columns"
  )
y_treino = dados_treino.ipca
x_treino = dados_treino.drop("ipca", axis = "columns")

In [6]:
# -------------------- ARIMA --------------------------
ajuste_arima = pmd.auto_arima(y = y_treino, seasonal = True, m = 12)
previsao_arima = ajuste_arima.predict(y_teste.shape[0])

# -------------------- XGBoost ------------------------------
ajuste_xgboost = xgb.XGBRegressor().fit(X = x_treino, y = y_treino)
previsao_xgboost = ajuste_xgboost.predict(x_teste)


resultados = pd.DataFrame( data = {"data": dados.data,"y": dados.ipca,
                                   "previsao_A": previsao_arima,
                                   "previsao_X": pd.Series(previsao_xgboost, index = previsao_arima.index)},
                                        index = dados.data)




In [7]:
fig = go.Figure() 
n=70
fig.add_trace(go.Scatter(x=dados.data.tail(n),y=resultados['y'].tail(n),mode='lines',name='Dados reais'))
fig.add_trace(go.Scatter(x=dados.data.tail(n),y=resultados['previsao_A'].tail(n),mode='lines', line_color='black',name='Previsão Arima'))
fig.add_trace(go.Scatter(x=dados.data.tail(n),y=resultados['previsao_X'].tail(n),mode='lines', line_color='red',name='Previsão XGBoost'))
fig.update_layout(
    title="Compara previsões ARIMA e XGBoost para o IPCA",
    xaxis_title="Ano", yaxis_title="Var. % mensal",
    xaxis_tickformat="%Y", xaxis_dtick="M24",
    legend_x=.75 ,  legend_y=1.3,)

fig.show()

In [8]:
from nixtlats import NixtlaClient
#https://dashboard.nixtla.io/
chave='nixt-rQMbAhxlC9xV5VtnmXCl79SY3R6s6K7lhbPoIHxI5dLUFRuOyTocvyy378PsxGg63WqtfqWat1uTUDrn'
nixtla_client = NixtlaClient( 
    api_key = chave)

In [9]:
nixtla_client.validate_api_key()

INFO:nixtlats.nixtla_client:Happy Forecasting! :), If you have questions or need support, please email ops@nixtla.io


True

In [10]:
# Produzir previsões
previsao_timegpt = nixtla_client.forecast(
  df = dados_treino.assign(data = dados_treino.index),
  # X_df = x_teste.assign(data = dados_teste.index),
  target_col = "ipca",
  time_col = "data",
  h = 12,
  freq = "MS"
  )


INFO:nixtlats.nixtla_client:Validating inputs...
INFO:nixtlats.nixtla_client:Preprocessing dataframes...
INFO:nixtlats.nixtla_client:Calling Forecast Endpoint...


In [11]:
resultados["previsao_IA"] = pd.Series(
      previsao_timegpt.TimeGPT.values,
      index = previsao_arima.index
      )

In [12]:
resultados['previsao_media']= resultados[['previsao_A','previsao_X','previsao_IA']].mean(axis=1)

In [13]:
fig = go.Figure() 
n=70
fig.add_trace(go.Scatter(x=dados.data.tail(n),y=resultados['y'].tail(n),mode='lines',name='Dados reais'))
fig.add_trace(go.Scatter(x=dados.data.tail(n),y=resultados['previsao_IA'].tail(n),mode='lines', line_color='black',name='Previsão Nixtla'))
fig.add_trace(go.Scatter(x=dados.data.tail(n),y=resultados['previsao_A'].tail(n),mode='lines', line_color='orange',name='Previsão Arima'))
fig.add_trace(go.Scatter(x=dados.data.tail(n),y=resultados['previsao_X'].tail(n),mode='lines', line_color='red',name='Previsão XGBoost'))
fig.add_trace(go.Scatter(x=dados.data.tail(n),y=resultados['previsao_media'].tail(n),mode='lines', line_color='yellow',name='Previsão média'))
fig.update_layout(
    title="Compara previsões Nixtla para o IPCA",
    xaxis_title="Ano", yaxis_title="Var. % mensal",
    xaxis_tickformat="%Y", xaxis_dtick="M24",
    legend_x=.75 ,  legend_y=1.3,)

fig.show()

In [14]:
# Calculando spreads
erro_arima = y_teste - previsao_arima
erro_xgboost = y_teste - previsao_xgboost
erro_timegpt = y_teste - previsao_timegpt.TimeGPT.values
erro_passeio_aleatorio = y_teste - y_treino.tail(12).values

# Calculando métrica RMSE
acuracia = pd.DataFrame(
  data = {
    "Modelo": ["ARIMA", "XGBoost", "Nixtla", "Passeio Aleatório"],
    "RMSE": (
      pd.Series((erro_arima.pow(2)).mean()).pow(0.5).values[0],
      pd.Series((erro_xgboost.pow(2)).mean()).pow(0.5).values[0],
      pd.Series((erro_timegpt.pow(2)).mean()).pow(0.5).values[0],
      pd.Series((erro_passeio_aleatorio.pow(2)).mean()).pow(0.5).values[0]
    )
  }
).assign(
  Modelo = lambda x: pd.Categorical(
    x.Modelo,
    categories = x.sort_values(by = "RMSE", ascending = False).Modelo.tolist()
    )
  )

#

In [15]:

# Criar o gráfico de barras
fig = go.Figure(data=[go.Bar(
    x=acuracia['Modelo'], 
    y=acuracia['RMSE']
)])

# Personalizar o gráfico
fig.update_layout(
    title="Desempenho de modelos na previsão do IPCA",
    xaxis_title="Modelo",
    yaxis_title="Erro Quadrático Médio",
    xaxis={'categoryorder':'total descending'},  # Ordenar barras do maior para o menor RMSE
    title_x=0.5,  # Centralizar o título
    
    # Adicionar nota como anotação
    annotations=[dict(
        x=0, 
        y=1.125, 
        xref="paper", 
        yref="paper", 
        text="Nota: valores para amostra de 12 meses até " + y_teste.index.max().strftime("%m/%Y"),
        showarrow=False
    )]
)

# Exibir o gráfico
fig.show()